## Guided Hunting - Detect potential network beaconing using Apache Spark via Azure Synapse

__Notebook Version:__ 1.1<br>
__Python Version:__ Python 3.8 - AzureML<br>
__Required Packages:__ Msticpy <br>
__Platforms Supported:__  Azure Machine Learning Notebooks connected to Azure Synapse Workspace
     
__Data Source Required:__ Yes

__Data Source:__ CommonSecurityLogs

__Spark Version:__ 3.1 or above
    
### Description
In this sample guided scenario notebook, we will demonstrate how to set up continuous data pipeline to store data into azure data lake storage (ADLS) and 
then hunt on that data at scale using distributed processing via Azure Synapse workspace connected to serverless Spark pool. 
Once historical dataset is available in ADLS , we can start performing common hunt operations, create a baseline of normal behavior using PySpark API and also apply data transformations 
to find anomalous behaviors such as periodic network beaconing as explained in the blog - [Detect Network beaconing via Intra-Request time delta patterns in Microsoft Sentinel - Microsoft Tech Community](https://techcommunity.microsoft.com/t5/azure-sentinel/detect-network-beaconing-via-intra-request-time-delta-patterns/ba-p/779586). 
You can use various other spark API to perform other data transformation to understand the data better. 
The output generated can also be further enriched to populate Geolocation information and also visualize using Msticpy capabilities to identify any anomalies. 
.<br>
<br>
*** Please run the cells sequentially to avoid errors.  Please do not use "run all cells". *** <br>

## Table of Contents
1. Pre-requisites
2. Data Preparation
3. Data Wrangling using Spark
4. Data Enrichment
5. Data Visualization
6. Data Ingestion to Sentinel as Custom Table
7. Conclusion



## Pre-requisites

### Set-up Synapse Spark Pool in Azure Machine Learning

This notebook requires Azure ML Spark Compute. If you are using it for the first time, follow the guidelines mentioned here 
[Attach and manage a Synapse Spark pool in Azure Machine Learning (preview)](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-synapse-spark-pool?tabs=studio-ui)

Once you have completed the pre-requisites, you will see AzureML Spark Compute in the dropdown menu for Compute. Select it and run any cell to start Spark Session. 
<br>Please refer the docs [Managed (Automatic) Spark compute in Azure Machine Learning Notebooks](https://learn.microsoft.com/en-us/azure/machine-learning/interactive-data-wrangling-with-apache-spark-azure-ml) for more detailed steps along with screenshots.


### Installing Libraries
Also, this notebook requires several libraries to be pre-installed prior to start Spark session. 
<br>In order to install any libraries, you can use a conda file to configure a Spark session.  Please save below file as conda.yml , check the Upload conda file checkbox. Then, select Browse, and choose the conda file saved earlier with the Spark session configuration you want.

```yaml
name: msticpy
channels:
- defaults
dependencies:
- bokeh
- numpy
- pip:
  - keyring>=13.2.1
  - msticpy[azure]>=2.3.1
```

### Set-up msticpyconfig.yaml to use with Azure ML with Managed Spark Pool

Once we've done above steps, we need to make sure we have configuration to tell MSTICPy how to connect to your sentinel workspace or any other supported data providers.

This configuration is stored in a configuration file (`msticpyconfig.yaml`).

Learn more...
Although you don't need to know these details now, you can find more information here:

- [MSTICPy Package Configuration](https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html)
- [MSTICPy Settings Editor](https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html)

If you need a more complete walk-through of configuration, we have a separate notebook to help you:
- [Configuring Notebook Environment](https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb)
- And for the ultimate walk-through of how to configure all your `msticpyconfig.yaml` settings see the [MPSettingsEditor notebook](https://github.com/microsoft/msticpy/blob/master/docs/notebooks/MPSettingsEditor.ipynb)
- The Azure-Sentinel-Notebooks GitHub repo also contains an template `msticpyconfig.yaml`, with commented-out sections that may also be helpful in finding your way around the settings if you want to dig into things by hand.


### Set environment variable MSTICPYCONFIG
Once you have your `msticpyconfig.yaml` ready in your Users directory, set the environment variable with the path to the file. 

Default file share is mounted and available to  Managed (Automatic) Spark compute. 
<br>Refer docs - [Accessing data on the default file share](https://learn.microsoft.com/en-us/azure/machine-learning/interactive-data-wrangling-with-apache-spark-azure-ml#accessing-data-on-the-default-file-share)

In below code, first we will generate the abspath of the current mount dir and append path where `msticpyconfig.yaml` is stored. 
<br>Execute the below cell to get the current path and then replace it in the next cell, also replace Username or path to `msticpyconfig.yaml`

In [ ]:
import os
from ipywidgets import widgets
# Provide AML Directory path to msticpyconfig.yaml e.g. /Users/ashwin/msticpyconfig.yaml
yaml_path = widgets.Text(description='AML Directory Path to msticpyconfig.aml:')
display(yaml_path)

In [ ]:
# Retrieve AML notebook mount path
abs_path = os.path.abspath('.')
msticpyconfig_path = f"{abs_path}{yaml_path.value}"
print(msticpyconfig_path)

In [ ]:
%env MSTICPYCONFIG = {msticpyconfig}

In [ ]:
# Load Python libraries that will be used in this notebook
import json
import os, uuid, sys
import IPython
import pandas as pd
from ipywidgets import widgets, Layout
from IPython.display import display, HTML
from pathlib import Path
from datetime import timedelta, datetime

import msticpy as mp
mp.refresh_config()

from msticpy.context.geoip import GeoLiteLookup
from msticpy.context.tilookup import TIl
from msticpy.nbtools import entityschema
from msticpy.sectools.ip_utils import convert_to_ip_entities
from msticpy.nbtools.foliummap import FoliumMap, get_map_center
from msticpy.data.uploaders.loganalytics_uploader import LAUploader
mp.check_version()
# mp.init_notebook(globals())

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 4, 4)

display(HTML("<h3>Starting Notebook setup...</h3>"))

# If not using Azure Notebooks, install msticpy with
# %pip install msticpy

WIDGET_DEFAULTS = {
    "layout": Layout(width="95%"),
    "style": {"description_width": "initial"},
}

#Set pandas options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

## Data Preparation
In this step, we will define several details associated with ADLS account and specify input date and lookback period to calculate baseline. Based on the input dates and lookback period , we will load the data.




In [ ]:
# Primary storage info
account_name = '<storage account name>' # fill in your primary account name
container_name = '<container name>' # fill in your container name
subscription_id = '<subscription if>' # fill in your subscription id
resource_group = '<resource group>' # fill in your resource groups for ADLS
workspace_name = '<Microsoft sentinel workspace name>' # fill in your workspace name
device_vendor = "Fortinet"  # Replace your desired network vendor from commonsecuritylogs

# Datetime and lookback parameters
end_date = "<enter date in the format yyyy-MM-dd e.g.2021-09-17>"  # fill in your input date
lookback_days = 21 # fill in lookback days if you want to run it on historical data. make sure you have historical data available in ADLS

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, col
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from datetime import timedelta, datetime, date

# Compiling ADLS paths from date string
end_date_str = end_date.split("-")
current_path = f"/y={end_date_str[0]}/m={end_date_str[1]}/d={end_date_str[2]}"

def generate_adls_paths(end_date, lookback_days, adls_path):
    endDate = datetime.strptime(end_date, '%Y-%m-%d')
    endDate = endDate - timedelta(days=1)
    startDate = endDate - timedelta(days=lookback_days)
    daterange = [startDate + timedelta(days=x) for x in range((endDate-startDate).days + 1)]

    pathlist = []
    for day in daterange:
        date_str = day.strftime('%Y-%m-%d').split("-")
        day_path = adls_path + f"/y={date_str[0]}/m={date_str[1]}/d={date_str[2]}"
        pathlist.append(day_path)

    return pathlist

adls_path = f'abfss://{container_name}@{account_name}.dfs.core.windows.net/WorkspaceResourceId=/subscriptions/{subscription_id}/resourcegroups/{resource_group}/providers/microsoft.operationalinsights/workspaces/{workspace_name}'
current_day_path = adls_path + current_path
historical_path = generate_adls_paths(end_date, lookback_days, adls_path)


### Load Current day

In this step, you will load the data based on the input date specified.

In [ ]:
try:
    current_df = (
        spark.read.option("recursiveFileLook", "true")
        .option("header", "true")
        .json(current_day_path)
    )

    current_df = ( 
            current_df
            .select(
                "TimeGenerated",
                "SourceIP",
                "SourcePort",
                "DestinationIP",
                "DestinationPort",
                "Protocol",
                "ReceivedBytes",
                "SentBytes",
                "DeviceVendor",
                    )
            .filter(F.col("DeviceVendor") == device_vendor)
                )
except Exception as e:
    print(f"Could note load the data due to error:{e}")

#Display the count of records
print(f"No of records loaded from the current date specified: {current_df.count()}")

### Load Historical data

You can also perform the analysis on all historical data available in your ADLS account. The notebook is currently configured to run only on current date specified in input. 

If you need to perform the same analysis on historical data, run the cell below and under Data Wrangling using Spark -> Filtering Data code cell, replace `current_df` with `historical_df` variable. 
<br>**Otherwise SKIP running below cell as it will result in an error if you do not have historical data**

In [ ]:
try:
    #Read Previous days data
    historical_df = (
        spark.read.option("recursiveFileLook", "true")
        .option("header", "true")
        .json(historical_path[-1])
    )
    historical_df = historical_df.select(
        "TimeGenerated",
        "SourceIP",
        "SourcePort",
        "DestinationIP",
        "DestinationPort",
        "Protocol",
        "ReceivedBytes",
        "SentBytes",
        "DeviceVendor",
    ).filter(F.col("DeviceVendor") == device_vendor)

    #Read all historical days data per day and union them together
    for path in historical_path[:-1]:
        daily_table = (
            spark.read.option("recursiveFileLook", "true")
            .option("header", "true")
            .json(path)
        )
        daily_table = daily_table.select(
            "TimeGenerated",
            "SourceIP",
            "SourcePort",
            "DestinationIP",
            "DestinationPort",
            "Protocol",
            "ReceivedBytes",
            "SentBytes",
            "DeviceVendor",
        ).filter(F.col("DeviceVendor") == device_vendor)
        historical_df = historical_df.union(daily_table)

except Exception as e:
    print(f"Could not load the data due to error:\n {e}")
    
#Display the count of records
print(f"\n No of records loaded from the lookback days specified: {historical_df.count()}")

## Data Wrangling using Spark

### Filtering data
In this step, we will prepare dataset by filtering logs to only destination as Public/external IPs. For this, we are using regex and rlike spark API to filter internal to external network traffic.

In [ ]:
PrivateIPregex = ("^127\.|^10\.|^172\.1[6-9]\.|^172\.2[0-9]\.|^172\.3[0-1]\.|^192\.168\.")
cooked_df = (current_df   # replace historical_df if you want to use historical data
            # use below filter if you have Palo Alto logs
            # .filter(
            #     (F.col("Activity") == "TRAFFIC")
            #     )
            .withColumn(
            "DestinationIsPrivate", F.col("DestinationIP").rlike(PrivateIPregex)
                        )
            .filter(F.col("DestinationIsPrivate") == "false")
            .withColumn("TimeGenerated", F.col("TimeGenerated").cast("timestamp"))
            )

cooked_df.show()

### Baseline data to filter known Source IP and Destination IPs

In this step, you can either analyze Historical data or current data to filter source IP and destination IP per defined criteria. 

In below example, we are filtering the Source IP which has daily event count more than the specified threshold.
<br> Also, you can filter the destination IPs whom very less source IPs are connecting. This will reduce false positives be filtering destination IPs which are commonly seen from internal systems which are likely benign.

In [ ]:
daily_event_count_threshold = 100  # Replace the threshold based on your environment or use default values 
degree_of_srcip_threshold = 25    # Replace the threshold based on your environment or use default values   

# Filtering IP list per TotalEventsThreshold
csl_srcip = (
        cooked_df.groupBy("SourceIP")
        .count()
        .filter(F.col("count") > daily_event_count_threshold)
        .orderBy(F.col("count"), ascending=False)
    )

# Filtering Destination IP list per Degree of Source IPs threshold
csl_dstip = (
        cooked_df.groupBy("DestinationIP")
        .agg(F.countDistinct("SourceIP").alias("DegreeofSourceIps"))
        .filter(F.col("DegreeofSourceIps") < degree_of_srcip_threshold)
    )

# Filtering IP list per Daily event threshold
baseline_df = (
        cooked_df.join(csl_srcip, ["SourceIP"])
        .join(csl_dstip, ["DestinationIP"])
        .select(
            "TimeGenerated",
            "SourceIP",
            "SourcePort",
            "DestinationIP",
            "DestinationPort",
            "Protocol",
            "ReceivedBytes",
            "SentBytes",
            "DeviceVendor",
        )
    )

baseline_df.show()

### Rank the datasets and Calculate PercentageBeaconing

In this step, we will use spark to wrangle the data by applying below transformations.
- Sort the dataset per SourceIP
- Calculate the time difference between First event and next event.
- Partition dataset per Source IP, Destination IP, Destination Port
- Window dataset into consecutive 3 to Calculate the Timedeltalistcount based on cluster of 1-3 timedelta events.
- Calculate percentagebeacon between TotalEventscount and Timedeltalistcount
- Apply thresholds to further reduce false positives

** SPARK References:**
- https://spark.apache.org/docs/latest/api/python/getting_started/quickstart.html
- https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#window

In [ ]:
time_delta_threshold = 15   # Replace thresholds in seconds interval between 2 successive events. min 10 to anything maximum.
percent_beacon_threshold = 75  # Replace thresholds in percentage. Max value can be 100.

# Serialize the dataset by sorting timegenerated and partition by SourceIP and WorkspaceId
w = (
        Window()
        .partitionBy(F.col("SourceIP"))
        .orderBy(F.col("TimeGenerated"))
    )

# Calculate new timestamp column of next event
csl_beacon_df = baseline_df.select(
        "*", lag("TimeGenerated").over(w).alias("prev_TimeStamp")
    ).na.drop()

# Calculate timedelta difference between previoud and next timestamp
timeDiff = F.unix_timestamp("TimeGenerated") - F.unix_timestamp("prev_TimeStamp")

# Add new column as timedelta
csl_beacon_df = csl_beacon_df.withColumn("Timedelta", timeDiff).filter(
        F.col("Timedelta") > time_delta_threshold
    )

csl_beacon_ranked = csl_beacon_df.groupBy(
        "DeviceVendor",
        "SourceIP",
        "DestinationIP",
        "DestinationPort",
        "Protocol",
        "Timedelta",
    ).agg(
        F.count("Timedelta").alias("Timedeltacount"),
        F.sum("SentBytes").alias("TotalSentBytes"),
        F.sum("ReceivedBytes").alias("TotalReceivedBytes"),
        F.count("*").alias("Totalevents"),
    )

w1 = (
        Window.partitionBy(
            "DeviceVendor",
            "SourceIP",
            "DestinationIP",
            "DestinationPort",
        )
        .orderBy(F.col("SourceIP").cast("long"))
        .rowsBetween(-2, 0)
    )

csl_beacon_df = (
        csl_beacon_ranked
        .join(csl_dstip, ["DestinationIP"])
        .withColumn("Timedeltalist", F.collect_list("Timedeltacount").over(w1))
        .withColumn(
            "Timedeltalistcount",
            F.expr("AGGREGATE(Timedeltalist, DOUBLE(0), (acc, x) -> acc + x)").cast(
                "long"
            ),
        )
        .withColumn(
            "Totaleventcount",
            F.sum("Timedeltalistcount").over(
                Window.partitionBy("SourceIP", "DestinationIP", "DestinationPort")
            ),
        )
        .withColumn(
            "Percentbeacon",
            (
                F.col("Timedeltalistcount")
                / F.sum("Timedeltalistcount").over(
                    Window.partitionBy(
                        "DeviceVendor",
                        "SourceIP",
                        "DestinationIP",
                        "DestinationPort",
                    )
                )
                * 100.0
            ),
        )
        .filter(F.col("Percentbeacon") > percent_beacon_threshold)
        .filter(F.col("Totaleventcount") > daily_event_count_threshold)
        .orderBy(F.col("Percentbeacon"), ascending=False)
    )


csl_beacon_df.show()

### Display results

In [ ]:
# Convert spark dataframe to pandas dataframe
df = csl_beacon_df.toPandas()
# Display results
df.head()

## Data Enrichment
In this section, we will enrich entities retrieved from network beaconing behavior such as IP information.
Types of Enrichment which will beneficial in perfoming investigation will be IP Geolcation , Whois Registrar information and ThreatIntel lookups.

For first time users, please refer `Getting Started Guide For Microsoft Sentinel ML Notebooks` and section [Create your configuration file](https://docs.microsoft.com/azure/sentinel/notebook-get-started#create-your-configuration-file) to create your `mstipyconfig`. 

### IP Geolocation Enrichment
In this step, we will use msticpy geolocation capabilities using maxmind database. You will need maxmind API key to download the database.

<div style="border: solid; padding: 5pt"><b>Note:</b>
You may see the GeoLite driver downloading its database the first time you run this.
</div>
<br>
<details>
    <summary>Learn more about MSTICPy GeoIP providers...</summary>
    <p>
    <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html >MSTICPy GeoIP Providers</a>
    </p>
</details>
<br>

In [ ]:
iplocation = GeoLiteLookup()

df = iplocation.df_lookup_ip(df, column="DestinationIP")
df.head()

### Whois registration enrichment
In this step, we can perform whois lokup on all public destination ips and populate additional information such as ASN. You can use this output to further filter known ASNs from the results.

In [ ]:
num_ips = len(df["DestinationIP"].unique())
print(f"Performing WhoIs lookups for {num_ips} IPs ", end="")
df["DestASN"] = df.apply(lambda x: get_whois_info(x.DestinationIP, True), axis=1)
df["DestASNFull"] = df.apply(lambda x: x.DestASN[1], axis=1)
df["DestASN"] = df.apply(lambda x: x.DestASN[0], axis=1)

#Display results
df.head()

### ThreatIntel Enrichment

In this step, we can perform threatintel lookup using msticpy and open source TI providers such as IBM Xforce, VirusTotal, Greynoise etc. 
Below example shows performing lookup on single IP as well as bulk lookup on all ips using IBM Xforce TI Provider. 
<br>You will need to register with IBM Xforce and enter API keys into `mstipyconfig.yaml`

<details>
    <summary>Learn more...</summary>
    <p>
    </p>
    <ul>
        <li>More details are shown in the <i>A Tour of Cybersec notebook features</i> notebook</li>
        <li><a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html >Threat Intel Lookups in MSTICPy</a></li>
        <li> To learn more about adding TI sources, see the TI Provider setup in the <i>A Getting Started Guide For Microsoft Sentinel ML Notebooks</i> notebook
    </ul>
</details>
<br>




In [ ]:
ti_lookup = TILookup()
# Perform lookup on single IOC
result = ti_lookup.lookup_ioc(observable="52.183.120.194", providers=["XForce"])
ti_lookup.result_to_df(result)

In [ ]:
# Flattening all the desnation IPs into comma separated list
ip_list = df['DestinationIP'].astype(str).values.flatten().tolist()

# Perform bulk lookup on all IPs with specified providers
ti_resp = ti_lookup.lookup_iocs(data=ip_list, providers=["AzSTI", "XForce"])
select_ti = browse_results(ti_resp, severities=['high','warning'])
select_ti

## Data Visualization
MSTICpy also includes a feature to allow you to map locations, this can be particularily useful when looking at the distribution of remote network connections or other events. Below we plot the locations of destination IPs observed in our results.

In [ ]:
# Convert our IP addresses in string format into an ip address entity
ip_entity = entityschema.IpAddress()
ip_list = [convert_to_ip_entities(i)[0] for i in df['DestinationIP'].head(10)]
    
# Get center location of all IP locaitons to center the map on
location = get_map_center(ip_list)
logon_map = FoliumMap(location=location, zoom_start=4)

# Add location markers to our map and dsiplay it
if len(ip_list) > 0:
    logon_map.add_ip_cluster(ip_entities=ip_list)
display(logon_map.folium_map)

## Data Ingestion to Sentinel as Custom Table
Finally, you can also upload the enriched events generated from this notebook as a custom table to Microsoft Sentinel. This will allow to correlate with more data sources available in sentinel and continue investigate or create incidents when malicious activity is confirmed.

In [ ]:
la_ws_id = widgets.Text(description='Workspace ID:')
la_ws_key = widgets.Password(description='Workspace Key:')
display(la_ws_id)
display(la_ws_key)

In [ ]:
# Instantiate data uploader for sentinel
la_up = LAUploader(workspace=la_ws_id.value, workspace_secret=la_ws_key.value, debug=True)
# Upload our DataFrame
la_up.upload_df(data=df, table_name='network_beaconing_anomalies')

## Conclusion

We originally started our hunting on very large datasets of firewall logs. Due to the sheer scale of data, we leveraged spark to load the data. 
<br>We then performed baselining on historical data and use it to further filter current day dataset. In the next step we performed various data transformation by using spark features such as partitioning, windowing, ranking dataset to find outbound network beaconing like behavior.
<br> In order to analyze this data further, we enrich IP entities from result dataset with additional information such as Geolocation, whois registration and threat intel lookups. 

Analysts can perform further investigation on selected IP addresses from enrichment results by correlating various data sources available. 
You can then create incidents in Microsoft Sentinel and track investigation in it.
